In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
from datetime import datetime
import numpy as np
from IPython.display import Image, display

In [2]:
%%bash

JULIA_VERSION="1.4.2"
JULIA_PACKAGES="DataFrames Feather ShiftedArrays RollingFunctions CSV Pipe Plots Flux Images Statistics"

JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
wget -nv $URL -O /tmp/julia.tar.gz
tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
rm /tmp/julia.tar.gz
for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'"'
done

Installing Julia package DataFrames...
Installing Julia package Feather...
Installing Julia package ShiftedArrays...
Installing Julia package RollingFunctions...
Installing Julia package CSV...
Installing Julia package Pipe...
Installing Julia package Plots...
Installing Julia package Flux...
Installing Julia package Images...
Installing Julia package Statistics...


2020-11-02 15:05:24 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.4/julia-1.4.2-linux-x86_64.tar.gz [99093958/99093958] -> "/tmp/julia.tar.gz" [1]
    Cloning default registries into `~/.julia`
    Cloning registry from "https://github.com/JuliaRegistries/General.git"
      Added registry `General` to `~/.julia/registries/General`
  Resolving package versions...
  Installed PooledArrays ──────────────── v0.5.3
  Installed SortingAlgorithms ─────────── v0.3.1
  Installed TableTraits ───────────────── v1.0.0
  Installed IteratorInterfaceExtensions ─ v1.0.0
  Installed DataAPI ───────────────────── v1.4.0
  Installed CategoricalArrays ─────────── v0.8.3
  Installed DataStructures ────────────── v0.18.8
  Installed DataFrames ────────────────── v0.21.8
  Installed Reexport ──────────────────── v0.2.0
  Installed OrderedCollections ────────── v1.3.2
  Installed StructTypes ───────────────── v1.1.0
  Installed InvertedIndices ───────────── v1.0.0
  Installed Tables ──────────

In [3]:

import os
os.environ['JULIA_NUM_THREADS'] = "4"

In [4]:
%%time
!pip install --quiet julia
import julia
from julia.api import Julia
julia.install()
jl = Julia(compiled_modules=False)  # cannot use precompiled packages with pyjulia on linux :-(
%load_ext julia.magic


Precompiling PyCall...
Precompiling PyCall... DONE
PyCall is installed and built successfully.


Initializing Julia interpreter. This may take some time...
CPU times: user 19.3 s, sys: 307 ms, total: 19.6 s
Wall time: 1min 4s


In [5]:
%%julia
using Pkg
Pkg.add("PyCall")

   Updating registry at `~/.julia/registries/General`
   Updating git-repo `https://github.com/JuliaRegistries/General.git`


  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]


In [6]:
%%julia
using PyCall
ImageDataGenerator = pyimport("keras").preprocessing.image.ImageDataGenerator
train = ImageDataGenerator(rescale = 1/255)
train_dir = "../input/dogs-cats-images/dataset/training_set/"
train_generator = train.flow_from_directory(train_dir, target_size = (224, 224), batch_size = 20, class_mode = "binary")

Found 8000 images belonging to 2 classes.


In [7]:
%%julia
using PyCall
ImageDataGenerator = pyimport("keras").preprocessing.image.ImageDataGenerator
train = ImageDataGenerator(rescale = 1/255)
train_dir = "../input/dogs-cats-images/dataset/test_set"
test_generator = train.flow_from_directory(train_dir, target_size = (224, 224), batch_size = 20, class_mode = "binary")

Found 2000 images belonging to 2 classes.


In [ ]:
%%julia
using Flux
function make_minibatch(X, Y, idxs)
    X_batch = Array{Float32}(undef, size(X[1])..., 1, length(idxs))
    for i in 1:length(idxs)
        X_batch[:, :, :, i] = Float32.(X[idxs[i]])
    end
    Y_batch = onehotbatch(Y[idxs], 0:9)
    return (X_batch, Y_batch)
end

In [ ]:
%%julia
img,label = test_generator
print(label)

In [13]:
%%julia
using Flux

2020-11-02 15:09:32.725667: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.2


########################################################################  99.3%############################### 100.0%-#O#- #   #                                                                     0.0%                                                 0.0%                                                                          0.2%                             0.6%  2.1%                                                 2.6%                                                                   5.8%  6.3%                                                           7.1%                                                         7.6%                                                           8.5%                  9.7% 10.2%                                       11.0%                                         11.5%                                         12.2%                                                        12.7%                  16.3%##########                                                      

######################################################################## 100.0%#################### 100.0%#=#=-#  #                                                                           0.0%                                                     0.0%                                       0.5%                                                         1.5%             5.0%  6.0%  7.3%#####                                                                     9.4%######                                                                   10.3% 13.4%             14.4%                                             18.5%19.4%                    21.6% 22.7%         23.5% 27.7%####################                                                     29.5%####################                                                     30.4%#####################                                                    31.2%######################                                                   33.1%##################

###################################################################       93.3%###### 100.0%#=#=-#  #                                                                                   0.0%  0.1%                            0.6%#######                                                              16.7%                                                     21.2%                                                  29.0%#############################                            64.3% 88.9%##### 100.0%


<h1>Model Building

In [18]:
%%julia
using Flux
model = Chain(
    Conv((3,3),3=>16,pad=(1,1),relu),
    MaxPool((2,2)),
    Conv((3,3),16=>32,pad=(1,1),relu),
    MaxPool((2,2)),
    #Conv((3,3),32=>32,pad= (1,1),relu),
    MaxPool((2,2)),
    Conv((3,3),32=>64,pad= (1,1),relu),
    MaxPool((2,2)),
    Conv((3,3),64=>128,pad= (1,1),relu),
    MaxPool((2,2)),
    Conv((3,3),128=>256,pad= (1,1),relu),
    MaxPool((2,2)),
    Conv((3,3),256=>512,pad= (1,1),relu),
    MaxPool((2,2)),
    #x->reshape(x,:,size(x,4)),
    Dense(4096,2),
    softmax,)

<PyCall.jlwrap Chain(Conv((3, 3), 3=>16, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 16=>32, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 32=>64, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 64=>128, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 128=>256, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 256=>512, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Dense(4096, 2), softmax)>

In [22]:
%%julia
imgs = [getarray(train_generator[i].) for i in 1:length(train_generator)]

RuntimeError: <PyCall.jlwrap (in a Julia function called from Python)
JULIA: type Tuple has no field img
Stacktrace:
 [1] getproperty(::Tuple{Array{Float32,4},Array{Float32,1}}, ::Symbol) at ./Base.jl:33
 [2] (::var"#31#32")(::Int64) at ./none:0
 [3] iterate at ./generator.jl:47 [inlined]
 [4] collect(::Base.Generator{UnitRange{Int64},var"#31#32"}) at ./array.jl:665
 [5] top-level scope at none:3
 [6] eval at ./boot.jl:331 [inlined]
 [7] eval at ./Base.jl:39 [inlined]
 [8] (::var"#29#30")(::PyObject, ::PyObject) at /opt/conda/lib/python3.7/site-packages/julia/pyjulia_helper.jl:91
 [9] (::PyCall.FuncWrapper{Tuple{PyObject,PyObject},var"#29#30"})(::PyObject, ::Vararg{PyObject,N} where N; kws::Base.Iterators.Pairs{Union{},Union{},Tuple{},NamedTuple{(),Tuple{}}}) at /root/.julia/packages/PyCall/BcTLp/src/callback.jl:61
 [10] (::PyCall.FuncWrapper{Tuple{PyObject,PyObject},var"#29#30"})(::PyObject, ::Vararg{PyObject,N} where N) at /root/.julia/packages/PyCall/BcTLp/src/callback.jl:61
 [11] i

In [25]:
%%julia
print(train_generator[0])

(Array{Float32}(undef,0,224,224,3), Float32[])

In [26]:
%%julia
using Pkg
Pkg.add("MappedArrays")
Pkg.add("FileIO")

  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
  [dbb5928d] + MappedArrays v0.2.2
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]
  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
  [5789e2e9] + FileIO v1.4.4
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]


In [31]:
%%julia
Pkg.add("ImageMagick")

  Resolving package versions...
  Installed Zstd_jll ──────── v1.4.5+2
  Installed JpegTurbo_jll ─── v2.0.1+3
  Installed libpng_jll ────── v1.6.37+6
  Installed Libtiff_jll ───── v4.1.0+2
  Installed ImageMagick_jll ─ v6.9.10-12+3
  Installed ImageMagick ───── v1.1.6
################################                                          45.7%#=#=-#  #                                                                                                                                                0.1%  2.9%                  11.7%####################################################################### 100.0%
######################################################                    37.0%##################################################### 100.0%#=#=-#  #                                                                     #                                                                          1.5%################## 100.0%
################################################                 

In [30]:
%%julia
using MappedArrays
using FileIO
using ImageMagick
function load_dataset(root)
    files = map(x->joinpath(root, x), readdir(root))
    return mappedarray(load, files)
end

root= "../input/dogs-cats-images/dataset/training_set/cats"
dataset = load_dataset(root)

Error encountered while loading "/kaggle/input/dogs-cats-images/dataset/training_set/cats/cat.1.jpg".

Fatal error:


ValueError: <PyCall.jlwrap (in a Julia function called from Python)
JULIA: ArgumentError: Package ImageMagick not found in current path:
- Run `import Pkg; Pkg.add("ImageMagick")` to install the ImageMagick package.

Stacktrace:
 [1] handle_error(::ArgumentError, ::File{DataFormat{:JPEG}}) at /root/.julia/packages/FileIO/2fEu2/src/error_handling.jl:82
 [2] handle_exceptions(::Array{Any,1}, ::String) at /root/.julia/packages/FileIO/2fEu2/src/error_handling.jl:77
 [3] load(::Formatted; options::Base.Iterators.Pairs{Union{},Union{},Tuple{},NamedTuple{(),Tuple{}}}) at /root/.julia/packages/FileIO/2fEu2/src/loadsave.jl:210
 [4] load at /root/.julia/packages/FileIO/2fEu2/src/loadsave.jl:187 [inlined]
 [5] #load#16 at /root/.julia/packages/FileIO/2fEu2/src/loadsave.jl:136 [inlined]
 [6] load at /root/.julia/packages/FileIO/2fEu2/src/loadsave.jl:136 [inlined]
 [7] mappedarray(::typeof(load), ::Array{String,1}) at /root/.julia/packages/MappedArrays/8HESW/src/MappedArrays.jl:61
 [8] load_dataset